In [1]:
from docxtpl import DocxTemplate, InlineImage, RichText
from html.parser import HTMLParser
from bs4 import BeautifulSoup as BS
from docx.shared import Mm, Cm
import PySimpleGUI as sg
import pandas as pd
import datetime
import os
import io
import re
import regex
import docx
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

pd.options.mode.chained_assignment = None

filepath = "Вывод/ОтчетПКПК_{}.docx".format(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M"))

file_path = "_сквозной_список_все_личные_счета.html"

blacklist = ['Глеб Черепанов', 'Черепанов Глеб', 'Андрей Богдашёв', 'Богдашёв Андрей', 'Андрей Богдашев', 'Богдашев Андрей',\
        ' Михаил Ковалёв', 'Ковалёв Михаил', 'Михаил Ковалев', 'Ковалев Михаил', 'Столярова Полина', 'Полина Столярова',\
        'Кристина Клюева', 'Клюева Кристина', 'Светлана Андреева', 'Андреева Светлана', 'Владислав Богдан', 'Богдан Владислав',\
        'Червоная Ольга', 'Александр Кожевников']

In [2]:
def window_string(key, text_size, input_size, tooltip=None):
    return ([sg.Text(key, size=(text_size,1)), sg.InputText(key=key, size=(input_size,1), tooltip=tooltip)])

def make_window():    
    layout1 = [
        [sg.HorizontalSeparator()],
        window_string('Номер договора', 26, 30),
        window_string('Число подписания договора', 26, 30, 'числом'),
        window_string('Месяц подписания договора', 26, 30, 'словом'),
        [sg.HorizontalSeparator()],
        window_string('Должность подписывающего', 26, 30, 'со стороны заказчика'),
        window_string('Краткое название заказчика', 26, 30),
        window_string('ФИО подписывающего', 26, 30, 'со стороны заказчика'),
        window_string('Договор заключён с ', 26, 30),
        window_string('Полная стоимость услуг', 26, 30, 'цифрами'),
        window_string('Полная стоимость услуг словами', 26, 30),
        [sg.HorizontalSeparator()],
        window_string('Дата проведения', 26, 30),
        window_string('Время начала', 26, 30, 'по местному времени'),
        window_string('Время окончания', 26, 30, 'по местному времени'),
        window_string('Адрес проведения', 26, 30),
        [sg.HorizontalSeparator()],
        [sg.Text('Тип игры', size=(26,1)), sg.Combo(['Отборочная игра Стартап-диплом', 'Коммерческая игра для студентов',
                'Коммерческая игра для взрослых', 'Бизнес-Дебют', 'Другое'],
                default_value='Отборочная игра Стартап-диплом', key='gametype', size=(30,1), readonly=True)],
        window_string('Название папки на диске', 26, 30),
        [sg.Checkbox('Дополнительный идентификатор 1', default=False, key='add_id_1')],
        [sg.Checkbox('Дополнительный идентификатор 2', default=False, key='add_id_2')],
        [sg.HorizontalSeparator()]
        ]
    
    layout2 = [
        [sg.HorizontalSeparator()],
        [sg.Text('Папка с данными игры', size=(20,1)),  sg.FolderBrowse('Путь', size=(4,1), target='folder_path'), sg.InputText(size=(30, 1), key='folder_path')],
        [sg.Text('Файл последнего уровня', size=(20,1)),  sg.FileBrowse('Путь', size=(4,1), target='comp_path'), sg.InputText(size=(30, 1), key='comp_path')],
        [sg.Text('Асессор комментарии', size=(20,1)),  sg.FileBrowse('Путь', size=(4,1), target='table_final'), sg.InputText(size=(30, 1), key='table_final')],
        [sg.Text('Асессор заключение', size=(20,1)),  sg.FileBrowse('Путь', size=(4,1), target='conclusion'), sg.InputText(size=(30, 1), key='conclusion')],
        [sg.ProgressBar(100, orientation='h', size=(40,20), key='progress_bar')],
        [sg.HorizontalSeparator()],
        [sg.Output(size=(50,14))],
        [sg.Submit('Запустить'), sg.Cancel('Выйти')],
        [sg.HorizontalSeparator()],
        ]
    
    tab_group_layout = [[sg.Tab('Строки', layout1), sg.Tab('Файлы', layout2)]]
    layout = [[sg.TabGroup(tab_group_layout, key='tab')]]
    
    return sg.Window('Мегаотчет v0.6', layout)

In [3]:
def html_read(result_list):
    player_name = result_list[1][7:] if result_list[1][-1] != ' ' else result_list[1][7:-1]
    table = result_list[result_list.index("Таблица транзакций") + 3 : result_list.index("Описание транзакций")]
    table = {table[i]: table.count(table[i]) for i in range(0, len(table), 2)}
    del result_list[:result_list.index("Описание транзакций") + 1]
    levels = [i for i, x in enumerate(result_list) if x[-2:] == ": "]
    result_list = [result_list[i].lower() + result_list[i + 1].lower() if i in levels else result_list[i] for i in range(len(result_list))]
    result_list = [result_list[i] for i in range(len(result_list)) if i - 1 not in levels]
    indexs = [i for i, x in enumerate(result_list) if x == "выполнена"]
    indexs.append(len(result_list))
    result_list = [result_list[indexs[n]:indexs[n + 1]] for n in range(len(indexs)-1)]
    result_list = [x if "уровень игры: " in ''.join(x) else x.append("уровень игры: " + str(table[x[1]])) or x for x in result_list]
    return (player_name, table, result_list)

def table_load():
    file_path_table = folder_path + file_path
    table = pd.read_html(file_path_table,encoding = 'utf-8')[0]
    table = table[table['Имя'].apply(lambda x: not x.startswith('Бот') and not x.startswith('Аватар') and not x in blacklist)]
    table['Баланс'] = table['Баланс'].apply(int)
    file = drive.ListFile({'q': "title = 'Nice2MeetYou' "}).GetList()[0]
    file.GetContentFile(file['title']+'.xlsx', mimetype = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
    nice = pd.read_excel(file['title']+'.xlsx', dtype = str, keep_default_na = False)
    return(table, nice)

def counter(values):
    class MyHTMLParser(HTMLParser):
        def handle_data(self, data):
            if not re.match(r'^\s*$', data):
                result_list.append(data)
        
    comps, top_players_count, trans_dict, trans_dict_deal = len(pd.read_html(comp_path, encoding='utf-8')[-1])-2, 0, {}, {}
    trans_all, trans_fire, trans_switch, trans_contr, trans_create, trans_close, trans_money, trans_deal, players_active = 0, 0, 0, 0, 0, 0, 0, 0, []
    comp_closed, comp_sold, comp_invested = set(), set(), set()
    for file_name in os.listdir(folder_path)[:]:
        if file_name.startswith("_игрок") and file_name.endswith(".html") and 'Бот' not in file_name and 'Аватар' not in file_name:
            result_list = []
            parser = MyHTMLParser()
            with io.open(folder_path+file_name, encoding = 'utf-8') as file:
                for line in file:
                    parser.feed(line)
            player_name, result_table, result_list = html_read(result_list)
            
            levels = set([[y[y.find(':')+2:] for y in x if y.startswith('уровень')][0] for x in result_list if 'заключить сделки' in ''.join(x)])
            if len(levels) >1 and player_name in list(table[table['Роль'] == 'Предприниматель']['Имя']):
                 top_players_count += 1
            
            trans_all = len(result_list)
            if 'автоматическое увольнение после окончания уровня' in result_table:
                trans_fire = result_table['автоматическое увольнение после окончания уровня']
            if 'cмена роли' in result_table:
                trans_switch = result_table['cмена роли']
            trans_contr = len([x for x in result_list if 'создать новый контракт' in ''.join(x)])
            trans_create = len([x for x in result_list if 'создать компанию' in ''.join(x)])
            trans_close = len([x for x in result_list if 'закрыть компанию' in ''.join(x)])
            trans_money = len([x for x in result_list if 'перевести талеры на счет компании' in ''.join(x)])
            trans_deal = int(len([x for x in result_list if 'заключить сделки' in ''.join(x)])/2)
            trans_right = trans_create+trans_close+trans_money+int((trans_all-trans_fire-trans_switch-trans_contr)/2)
            trans_dict.update({player_name: trans_right})
            trans_dict_deal.update({player_name: trans_deal})
            
            levels = set([[y[y.find(':')+2:] for y in x if y.startswith('уровень')][0] for x in result_list])
            if any(num in levels for num in ['4', '5']):
                players_active.append(player_name)
            
            comp_closed.update([[y[y.find(':')+2:] for y in x if y.startswith('название')][0] for x in result_list if 'закрыть компанию' in ''.join(x)])            
            comp_sold.update([[y[y.find(':')+2:] for y in x if y.startswith('компания продавец')][0] for x in result_list if 'продать компанию' in ''.join(x)])
            comp_invested.update([[y[y.find(':')+2:] for y in x if y.startswith('в компании')][0] for x in result_list if 'инвестировать' in ''.join(x)])
            #print(trans_create, trans_close, trans_money, trans_all, trans_fire, trans_switch, trans_contr, trans_dict[player_name])
    top_players_part = round(100*top_players_count/len(table))
    top_players_level = 'высокий' if top_players_part >= 0.12 else 'средний'
    eng_active = len(table[(table['Роль']=='Инженер') & (table['Имя'].isin(players_active))])
    ent_active = len(table[(table['Роль']=='Предприниматель') & (table['Имя'].isin(players_active))])
    
    trans_right = sum(trans_dict.values())
    trans_deal = sum(trans_dict_deal.values())
    
    #print(trans_right, trans_deal)
    for key, value in zip(['top_players_part', 'top_players_level', 'comps', 'eng_active', 'ent_active', 'trans_right', 'trans_deal', 'comp_closed', 'comp_sold', 'comp_invested'],
                          [top_players_part, top_players_level, comps, eng_active, ent_active, trans_right, trans_deal, len(comp_closed), len(comp_sold), len(comp_invested)]):
        values[key] = value
    #print(values)
    return(trans_dict)

def search(names, name):
    try:
        name1, name2 = name.split()
        result = [name for name in names if name1 in name.split(' ') and name2 in name.split(' ')]
        if result:
            return max(result)
        else:
            return(name)
    except:
        return(name)

def drive_folder_create(parent_id, folder_name):
    file = drive.CreateFile({'title': folder_name,
        "parents": [{"id": parent_id}],
        "mimeType": "application/vnd.google-apps.folder"})
    file.Upload()
    permission = file.InsertPermission({
                        'type': 'anyone',
                        'value': 'anyone',
                        'role': 'reader'})
    return(file)

def drive_file_create(file_name, parent_id, file_path):
    file = drive.CreateFile({
        'title': file_name, 
        "parents": [{"id": parent_id}]
        })
    file.SetContentFile(file_path)
    file.Upload()
    permission = file.InsertPermission({
                    'type': 'anyone',
                    'value': 'anyone',
                    'role': 'reader'})
    return(file['alternateLink'])

def counter_new(table):
    folder = drive_folder_create(folder_root_id, values['Название папки на диске'] if values['Название папки на диске'] != '' else 'Новая папка')
    folder_id, folder_link = folder['id'], folder['alternateLink']
    values['folder_link'] = folder_link
    folder_id_1 = drive_folder_create(folder_id, '1й эшелон')['id']
    folder_id_2 = drive_folder_create(folder_id, '2й эшелон')['id']
    folder_id_3 = drive_folder_create(folder_id, '3й эшелон')['id']
    folder_id_4 = drive_folder_create(folder_id, '4й эшелон')['id']
    doc_conclusion = docx.Document(values['conclusion'])
    fullText = []
    for para in doc_conclusion.paragraphs:
        fullText.append(para.text)
    if 'Пример общего заключения асессора:' in fullText:
        fullText = fullText[:fullText.index('Пример общего заключения асессора:')]
    values['conclusion'] = fullText[1:]
    
    values['last_level'] = values['comp_path'][values['comp_path'].find('.html')-1]
    players_last_level_active = 0
    status_final_dict = {}
    for file_name in os.listdir(folder_path)[:]:
        if file_name.startswith("_игрок") and file_name.endswith(".html") and all([name not in file_name for name in ['Бот', 'Аватар', 'Акционер']]):
            status_final_dict[file_name] = 'Выбыший'
            transactions = pd.read_html(folder_path+file_name,encoding = 'utf-8')[0]
            if int(values['last_level']) in set(transactions['Уровень']):
                status_final_dict[file_name] = 'Финалист'
                players_last_level_active += 1
    values['players_last_level_active'] = players_last_level_active
    table['Файл'] = table['Имя'].apply(lambda x: [file_name for file_name in os.listdir(folder_path) if x in file_name][0])
    table['Статус'] = table['Файл'].apply(lambda x: status_final_dict[x])
    table_final = pd.read_excel(values['table_final'])
    table_1 = pd.merge(table, table_final, left_on="Контакт", right_on="Email", how="right")
    table_2 = pd.merge(table, table_final, left_on="Имя", right_on="ФИО", how="right")
    table_1[table_1['Контакт'].isnull()] = table_2.iloc[table_1[table_1.isnull().any(axis=1)].index]
    table_final = table_1
    table = table[~table['Контакт'].isin(table_final['Email'])]
    table = table[~table['Имя'].isin(table_final['ФИО'])]
    table['Эшелон'] = table['Статус'].apply(lambda x: 3 if x == 'Финалист' else 4)
    table_final = pd.concat([table_final, table[table['Эшелон'] == 3], table[table['Эшелон'] == 4]])
    table_final.reset_index(drop=True, inplace=True)
    table_final['№'] = table_final.index + 1
    table_final.fillna('?', inplace=True)
    table_final['Баланс'] = table_final['Баланс'].apply(lambda x: int(x) if isinstance(x, float) else x)
    values['players_all'] = len(table_final)
    
    col_labels = ['№', 'Имя', 'Контакт', 'Роль', 'Баланс', 'Статус']
    col_labels_final = ['№', 'Имя', 'Контакт', 'Роль', 'Баланс', 'Статус']
    if values['gametype'] == "Отборочная игра Стартап-диплом" and values['add_id_1'] and values['add_id_2']:
        values['key'] = 2
        col_labels_final.insert(2, 'Факультет')
        table_final['Факультет'] = ''
        col_labels_final.insert(3, 'Курс')
        table_final['Курс'] = ''
    elif values['gametype'] == "Отборочная игра Стартап-диплом" and values['add_id_1']:
        values['key'] = 1
        col_labels_final.insert(2, 'Факультет')
        table_final['Факультет'] = ''
    elif values['gametype'] == "Отборочная игра Стартап-диплом" and values['add_id_2']:
        values['key'] = 1
        col_labels_final.insert(2, 'Курс')
        table_final['Курс'] = ''
    else:
        values['key'] = 0
    values['col_labels_final'] = col_labels_final
    col_labels = col_labels + ['Комментарий', 'Рекомендация для участника', 'Эшелон']
    
    doc = DocxTemplate("Ввод/тело_отчёта_шаблон {}.docx".format(values['key']))
    for key in list(values.keys()):
        if isinstance(key, str) and isinstance(values[key], str):
            key_new = key.replace(" ", "_")
            values[key] = values[key].replace("Нет", "")
            values[key_new] = values.pop(key)
    values['map_2'] = InlineImage(doc, folder_path+'generated/graph/new_generate_level2.png', Cm(10))
    values['map_3'] = InlineImage(doc, folder_path+'generated/graph/new_generate_level3.png', Cm(10))
    values['map_4'] = InlineImage(doc, folder_path+'generated/graph/new_generate_level4.png', Cm(10))
    values['map_5'] = InlineImage(doc, folder_path+'generated/graph/new_generate_level5.png', Cm(10))
    
    echelon_1, echelon_2, echelon_3, echelon_4 = [], [], [], []
    echelon_dict = {1: echelon_1, 2: echelon_2, 3: echelon_3, 4: echelon_4}
    folder_dict = {1: folder_id_1, 2: folder_id_2, 3: folder_id_3, 4: folder_id_4}
    
    for rating, name, email, role, score, status_final, comment, recomm, echelon in table_final[col_labels].values[:]:
        try:
            name = str(name)
        except:
            pass
        for file_name in os.listdir(folder_path):
            if name in file_name:
                status, phone, univer_f, univer_c = '-', '-', '-', '-'
                transactions = pd.read_html(folder_path + file_name)[0]
                if 'продать компанию' in transactions['Транзакция'].values:
                    status = 'Продал компанию'
                elif 'инвестировать' in transactions['Транзакция'].values:
                    status = 'Получил инвестиции'
                else:
                    try:
                        max_level = str(max(set(transactions['Уровень'])))
                    except:
                        max_level = '0'
                    if max_level == values['last_level']:
                        status = 'Дошёл до конца игры'
                    else:
                        status = 'Выбыл на {} уровне'.format(max_level)

                if pd.isna(email):
                    email = '-'   
                
                name = search(nice['name'], name)
                try:
                    univer_c = max(nice[nice['ma_email'].str.lower() == email.lower()]['univer'], key=len)
                    table_final.at[table_final[table_final['Контакт']==email].index[0], 'Курс'] = univer_c
                    univer_f = max(nice[nice['ma_email'].str.lower() == email.lower()]['univer_2'], key=len)
                    table_final.at[table_final[table_final['Контакт']==email].index[0], 'Факультет'] = univer_f
                except:
                    pass
                """
                try:
                    univer_c = max(nice[nice['name'] == name]['univer'], key=len)
                    table_final.at[table_final['Контакт']==email, 'Курс'] = univer_c
                    univer_f = max(nice[nice['name'] == name]['univer_2'], key=len)
                    table_final.at[table_final['Контакт']==email, 'Факультет'] = univer_f
                    if email == '-':
                        email = max(nice[nice['name'] == name]['ma_email'], key=len)
                    if phone == '-':
                        phone = max(nice[nice['name'] == name]['phone'], key=len)
                except:
                    try:
                        if phone == '-':
                            name = max(max(nice[nice['ma_email'].apply(str.lower) == email.lower()]['name'], key=len), name, key=len)
                            univer_c = max(nice[nice['ma_email'].apply(str.lower) == email.lower()]['univer'], key=len)
                            table_final.at[table_final['Контакт']==email, 'Курс'] = univer_c
                            univer_f = max(nice[nice['ma_email'].apply(str.lower) == email.lower()]['univer_2'], key=len)
                            table_final.at[table_final['Контакт']==email, 'Факультет'] = univer_f
                            phone = max(nice[nice['ma_email'].apply(str.lower) == email.lower()]['phone'], key=len)
                    except:
                        pass
                """
                    
                if values['gametype'] != "Отборочная игра Стартап-диплом" or not values['add_id_1']:
                    univer_f = ''
                if values['gametype'] != "Отборочная игра Стартап-диплом" or not values['add_id_2']: 
                    univer_c = ''

                doc_disk = DocxTemplate("Ввод/Профиль {}.{} эшелон - диск.docx".format(echelon, values['key']))
                with open(folder_path + file_name, 'r', encoding='utf-8') as f:
                    html = f.read()
                soup = BS(html, features='lxml')
                for image_link in soup.find_all('img'):
                    if image_link['src'][:9] == 'generated':
                        player_map = InlineImage(doc, folder_path + image_link['src'][:image_link['src'].find('?')], Cm(10))
                        player_map_disk = InlineImage(doc_disk, folder_path + image_link['src'][:image_link['src'].find('?')], Cm(10))
                item = {'name': name, 'email': email, 'date': values['Дата_проведения'], 'role': role, 'score': score, 'rating': rating, 'map': player_map_disk, 'status': status,
                        'transactions': len(transactions), 'comment': comment, 'recomm': recomm, 'phone': phone, 'univer': univer_f, 'univer_2': univer_c, 'key': values['key']}
                doc_disk.render({'item': item})
                doc_disk.save('Вывод/'+name+'.docx')
                link = drive_file_create(name, folder_dict[echelon], 'Вывод/'+name+'.docx')
                item['link'] = link
                item['map'] = player_map
                echelon_dict[echelon].append(item)
                window['progress_bar'].update(round(100*rating/len(table_final)))
    values['table_final'] = table_final[values['col_labels_final']].values
    values['table_1'] = table_final[table_final['Эшелон'] == 1][values['col_labels_final']].values
    values['table_2'] = table_final[table_final['Эшелон'] == 2][values['col_labels_final']].values
    values['table_3'] = table_final[table_final['Эшелон'] == 3][values['col_labels_final']].values
    values['echelon_1'], values['echelon_2'], values['echelon_3'] = echelon_1, echelon_2, echelon_3
    values['echelon_1_len'], values['echelon_2_len'], values['echelon_3_len'] = len(echelon_1), len(echelon_2), len(echelon_3)
    
    doc.render(values)
    doc.save(filepath)

    return(table_final)

In [5]:
sg.theme('Reddit')
window = make_window()

gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)
root_id = drive.ListFile({'q': "title = 'Продукты'"}).GetList()[0]['id']
folder_root_id = drive.ListFile({'q': "title = 'Отчеты' and '%s' in parents" % root_id}).GetList()[0]['id']

while True:
    event, values = window.read()
    if event in (None, 'Exit', 'Выйти'):
        break
    if event == 'Запустить':
        folder_path = values['folder_path']+'/'
        comp_path = values['comp_path']
        table, nice = table_load()
        trans_dict = counter(values)
        table_final = counter_new(table)
        print('Файл создан: '+filepath)
window.close()

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=922428539074-3lcp6a62nd735nvdh38lj548n223snd0.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [ ]:
def html_read(result_list):
    player_name = result_list[1][7:] if result_list[1][-1] != ' ' else result_list[1][7:-1]
    table = result_list[result_list.index("Таблица транзакций") + 3 : result_list.index("Описание транзакций")]
    table = {table[i]: table.count(table[i]) for i in range(0, len(table), 2)}
    del result_list[:result_list.index("Описание транзакций") + 1]
    levels = [i for i, x in enumerate(result_list) if x[-2:] == ": "]
    result_list = [result_list[i].lower() + result_list[i + 1].lower() if i in levels else result_list[i] for i in range(len(result_list))]
    result_list = [result_list[i] for i in range(len(result_list)) if i - 1 not in levels]
    indexs = [i for i, x in enumerate(result_list) if x == "выполнена"]
    indexs.append(len(result_list))
    result_list = [result_list[indexs[n]:indexs[n + 1]] for n in range(len(indexs)-1)]
    result_list = [x if "уровень игры: " in ''.join(x) else x.append("уровень игры: " + str(table[x[1]])) or x for x in result_list]
    return (player_name, table, result_list)

In [ ]:
import pandas as pd
import os, io, re
from html.parser import HTMLParser

class MyHTMLParser(HTMLParser):
    def handle_data(self, data):
        if not re.match(r'^\s*$', data):
            result_list.append(data)

for file_name in os.listdir('backup/data'):
    try:
        if file_name.startswith("_игрок") and file_name.endswith(".html") and 'Бот' not in file_name and 'Аватар' not in file_name:
            result_list = []
            parser = MyHTMLParser()
            with io.open('backup/data/'+file_name, encoding = 'utf-8') as file:
                for line in file:
                    parser.feed(line)
            player_name, result_table, result_list = html_read(result_list)
    except:
        print(file_name)
        os.remove('backup/data/'+file_name)